---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [7]:
import numpy as np 

K = 50                      # strike price
r = 0.03                    # risk-free rate per period
q = 0.04                    # dividend yield
sigma = 0.40                # volatility per year
T = 3                       # years to maturity
N = 40                      # number of time steps in tree

# CALCULATING OPTION VALUES

To speed up the calculation of an American option value, we use the Black-Scholes formula at the penultimate date rather than the binomial tree to calculate the value of the option if not exercised at that date.  This is sensible, because there are no further opportunities to exercise after that date except at maturity, so the option is European if not exercised at the penultimate date.  

In [8]:
import numpy as np
from scipy.stats import norm

def BS(S, time_to_mat, kind):
    S = np.maximum(S, 1.0e-6)
    d1 = np.log(S / K) + (r - q + 0.5 * sigma ** 2) * time_to_mat
    d1 /= sigma * np.sqrt(time_to_mat)
    d2 = d1 - sigma * np.sqrt(time_to_mat)
    if kind == "call":
        return (
            np.exp(-q * time_to_mat) * S * norm.cdf(d1) - 
            np.exp(-r * time_to_mat) * K * norm.cdf(d2)
        )
    else:
        return (
            np.exp(-r * time_to_mat) * K * norm.cdf(-d2) - 
            np.exp(-q * time_to_mat) * S * norm.cdf(-d1)
        )
   

def American(S, kind):
    
    intrinsic = lambda x: (x - K if kind == "call" else K - x)
    dt = T / N
    up = np.exp(sigma * np.sqrt(dt))
    down = 1 / up
    prob = (np.exp((r - q) * dt) - down) / (up - down)
    discount = np.exp(-r * dt)

    # Black-Scholes at penultimate date
    x = S * up ** np.arange(N - 1, -N - 1, -2)
    v = np.maximum(intrinsic(x), BS(x, dt, kind))

    # step backward in the tree until date 0
    for n in range(N-2, -1, -1):
        x = S * up ** n
        v[0] = np.maximum(
            intrinsic(x), 
            discount * (prob * v[0] + (1 - prob) * v[1])
        )
        for i in range(1, n + 1):
            x *= down * down
            v[i] = np.maximum(
                intrinsic(x), 
                discount * (prob * v[i] + (1 - prob) * v[i + 1])
            )
    return v[0]

# CREATE FIGURES

In [17]:
import plotly.graph_objects as go 

grid = np.arange(1, 101)
figs = []
for kind in ["call", "put"]:

    # European
    fig = go.Figure(
        go.Scatter(
            x=grid, 
            y=[BS(S, T, kind) for S in grid], 
            mode="lines", 
            hovertemplate="European value = $%{y:.2f}",
            name="European"
        )
    )
  
    # American
    fig.add_trace(
        go.Scatter(
            x=grid,
            y=[American(S, kind) for S in grid],
            mode="lines",
            hovertemplate="American value = $%{y:.2f}",
            name="American",
            legendrank=1
        )
    )

    # intrinsic value
    fig.add_trace(
        go.Scatter(
            x=grid,
            y=np.maximum(grid-K, 0) if kind=="call" else np.maximum(K-grid, 0),
            mode="lines",
            hovertemplate="intrinsic value = $%{y:.2f}",
            name="intrinsic value",
        )
    )      
    
    fig.update_layout(
        xaxis_title="Underlying Price",
        yaxis_title="Option Value",
        xaxis_tickformat=".0f",
        yaxis_tickformat=".0f", 
        xaxis_tickprefix="$",
        yaxis_tickprefix="$",
        template="plotly_white",
        hovermode="x unified",
        legend=dict(
            xanchor="left" if kind=="call" else "right",
            yanchor="top",
            x=0.01 if kind=="call" else 0.99,
            y=0.99
        ),
    )
    figs.append(fig)

# CALL FIGURE

In [18]:
figs[0].show()

# PUT FIGURE

In [19]:
figs[1].show()